## <center>Підготовка та аналіз даних</center>
## <center>Лабораторна робота №2</center>
#### <center>ФБ-25 Савченко Єлизавета</center>

In [7]:
!pip install pandas --break-system-packages


Defaulting to user installation because normal site-packages is not writeable


Імпортуємо бібліотеки

In [8]:
import datetime
import requests
import urllib.request
import pandas as pd
import os

In [9]:
"Cкачуємо індекси для всіх областей. Додаємо в назву дату та час скачування"

'Cкачуємо індекси для всіх областей. Додаємо в назву дату та час скачування'

In [ ]:
if not os.path.exists('vhi'):
    os.mkdir('vhi')
    print(f'The folder is created')

for ids in range(1, 28):
    url = f"https://www.star.nesdis.noaa.gov/smcd/emb/vci/VH/get_TS_admin.php?country=UKR&provinceID={ids}&year1=1981&year2=2024&type=Mean"
    response = requests.get(url)

    if response.status_code != 200:
        print(f"Download complete")
        break

    date_now = datetime.datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
    vhi_url = urllib.request.urlopen(url)
    file_name = f'vhi/vhi_id_{ids}_{date_now}.csv'
    out = open(file_name, 'wb')
    out.write(vhi_url.read())
    out.close()
    print(f"VHI from id {ids} was downloaded at {date_now}")

Даємо імена стовбцям та зчитуємо файли у фрейм

In [ ]:
folder = 'vhi'
files = os.listdir(folder)
# ids = 0
df_all = pd.DataFrame()

for file_name in files:
    # ids += 1
    
    headers = ['Year', 'Week', 'SMN', 'SMT', 'VCI', 'TCI', 'VHI', 'empty']
    df = pd.read_csv(f'{folder}/{file_name}', header=1, names=headers, skiprows=1)[:-1]
    df = df.drop(df.loc[df['VHI'] == -1].index)
    df['area'] = int(file_name.split("_")[-2])
    # print(df.head(), '\n')
    # df_all = pd.concat([df_all, df], ignore_index=True)
    df = df.drop(columns=['empty'])
    df_all = df_all.dropna(axis=1, how='all')
    df_all = pd.concat([df_all, df]).drop_duplicates().reset_index(drop=True)
    
# zm = df_all.isna().sum()
# print(zm)
print(df_all, '\n')

Змінюємо індекси областей

In [ ]:
dict_areas = {1: 22, 2: 24, 3: 23, 4: 25, 5: 3, 6: 4, 7: 8, 8: 19, 9: 20, 10: 21, 11: 9, 13: 10, 14: 11, 15: 12,
              16: 13, 17: 15, 18: 14, 19: 16, 21: 17, 22: 18, 23: 6, 24: 1, 25: 2, 26: 7, 27: 5}
df_all["area"] = df_all["area"].replace(dict_areas)
df_all.to_csv(f'vhi/df_all.csv', index=False)

print(df_all)

Ряд VHI для області за вказаний рік, пошук екстремумів (min та max);

In [ ]:
def vhi(area, year):
    filtered_df = df_all[(df_all["area"] == area) & (df_all["Year"] == year)]
    return filtered_df['VHI']

def vhi_min(area, year):
    filtered_df = df_all[(df_all["area"] == area) & (df_all["Year"] == year)]
    return filtered_df['VHI'].min()

def vhi_max(area, year):
    filtered_df = df_all[(df_all["area"] == area) & (df_all["Year"] == year)]
    return filtered_df['VHI'].max()

df_all['Year'] = df_all['Year'].astype(int)


In [ ]:
vhi(6, 2005)

In [ ]:
vhi_min(6, 2005)


In [ ]:
vhi_max(6, 2005)

Ряд VHI за вказаний діапазон років для вказаних областей

In [ ]:
def vhi_range(year_min, year_max, areas):
    if not isinstance(areas, list) or not areas:
        print('Empty or not a list')
        return
    filtered_df = df_all[(df_all['Year'] >= year_min) & (df_all['Year'] <= year_max) & (df_all['area'].isin(areas))]
    return filtered_df[['Year', 'VHI', 'area']]


In [ ]:
vhi_range(2000, 2004, [5, 2, 3])

Роки, протягом яких екстремальні посухи торкнулися більше вказаного відсотка областей

In [ ]:
def extreme_droughts(procent):
    df_drouhts = df_all[(df_all['VHI'] <= 15) & (df_all['VHI'] != -1)]
    grouped = df_drouhts.groupby('Year')['area'].nunique()
    result = grouped[grouped > (25 * procent / 100)].reset_index()
    return result

In [ ]:
extreme_droughts(5)

І так само для помірних посух

In [ ]:
def moderate_droughts(procent, min=15, max=40):
    df_drouhts = df_all[(df_all['VHI'] >= min) & (df_all['VHI'] <= max)]
    grouped = df_drouhts.groupby('Year')['area'].nunique()
    count_areas = df_all['area'].nunique()
    result = grouped[grouped > (count_areas * procent / 100)].reset_index()
    return result

In [ ]:
moderate_droughts(10)